# TP Chapitre - Entraînez-vous : implémentez une validation croisée

This the notebook where I constructed the method, step by step, so there are probably repeated parts of code.

In [37]:
data_path = "~/Documents/openclassroom/Fomation_ingenieur_ML/data/"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## sklearn module : 
from sklearn import model_selection 
from sklearn import preprocessing
from sklearn import neighbors


In [74]:
df = pd.read_csv(data_path+'winequality-red+.csv', sep=";")

## DESIGN AND SAMPLING : 
X = df.drop("quality", axis = 1, inplace = False)
threshold = np.floor(df.quality.mean())
y = np.where(df.quality < threshold, 0,1) 
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

#### comment je créée mes conteneurs pour extraire les données train/test ?
## PAR UN DICT GLOBAL ?
# original_data_split={"X_train":X_train,"X_test":X_test,
#                      "y_train":y_train, "y_test":y_test}
# original_data_split["X_train"]

# class DataSplit:
#    def __init__(self):
#     X_train = self.X_train
#     y_train = self.y_train
#     X_test = self.X_test
#     y_test = self.y_test
    
    
## EXTRACT X_train, X_test FROM X AND train_index :    
def extract_Xsplitted_data(X, train_index):    
#     train_index = X_train.index
    test_index = [(index not in train_index) for index in X.index]
    X_train = X.loc[train_index]
    X_test = X.loc[test_index]
    return(X_train, X_test)

## STANDARDIZATION : 
def normalize_from_split(X, train_index):
    X_train, X_test = extract_Xsplitted_data(X, train_index)
    
    my_normalizer = preprocessing.StandardScaler().fit(X_train)
    X_train_std = pd.DataFrame(my_normalizer.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_test_std = pd.DataFrame(my_normalizer.transform(X_test), index = X_test.index, columns = X_test.columns)

    return(X_train_std, X_test_std)


###### LIST OF ARGUMENTS AT THE END 
clf_meth = neighbors.KNeighborsClassifier()

param_grid = {"n_neighbors" : np.arange(2,15,1)}

cv = 5

###### WHAT IT DOES : 
1. Create "cv"(=5 for ex) folds 
    
    1.1 Separate the observations in 5 parts that respect the global distrib (folds)

    1.2 Loop for fold_f in 1:5, 
            fit the clf_meth model on the Xcv_train = X_train[all fold - fold_f],
    
            loop for params in param_grid.keys() : 
            
                fit clf_meth(Xcv_train, ycv_train
            
            save results for prediction on Xcv_test = X_train[fold_f] 

2. define class methods : 
    
    2.1 cv_results_ : save mean results of models in the dict param_grid.key()
    
    2.2 best_params_ : the better parameter set in param_grid, in term of accuracy
    ... many other methods, see https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

## 1. With the Kflod function in scikit-learn :
In the chapter "Mettez en place un cadre de validation croisée", they explain all the approach of the Cross Validation, from the separation in kflod, with the "same" classification distribution (using sklearn.model_selection.KFold or sklearn.model_selection.StratifiedKFold). So, let's first implement the second part, that is where the hyperparameters are fitted. 

In [81]:
## DESIGN AND SAMPLING : 
X = df.drop("quality", axis = 1, inplace = False)
threshold = np.floor(df.quality.mean())
y = np.where(df.quality < threshold, 0,1) 
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

from sklearn.model_selection import KFold
##TODO : attention, ici le X_train devient le "X" pour la CV 
CV_X = X_train
CV_y = y_train 

kf = KFold(n_splits = 5)
# kf.get_n_splits(argX)

for train_range_index, test_range_index in kf.split(CV_X) : 
#     print("TRAIN:", CV_train_index, "TEST:", CV_test_index)
    train_index = CV_X.index[train_range_index]
#     test_index = CV_X.index[test_range_index] 
#     CV_X_train, CV_X_test = CV_X.loc[train_index], CV_X.loc[test_index]
#     CV_y_train, CV_y_test = y[train_index], y[test_index]
    
    ## NORMALIZE :
    CV_X_train, CV_X_test = normalize_from_split(CV_X, train_index)
    
    ## FIT KNN on X_train AND PREDICT ON X_test :  

In [76]:
X = CV_X

CV_X.index[train_range_index]
# X_train, X_test = extract_Xsplitted_data(X, train_index)

# my_normalizer = preprocessing.StandardScaler().fit(X_train)
# X_train_std = pd.DataFrame(my_normalizer.transform(X_train), index = X_train.index, columns = X_train.columns)
# X_test_std = pd.DataFrame(my_normalizer.transform(X_test), index = X_test.index, columns = X_test.columns)


Int64Index([ 120, 1482,  211,  879,  609, 1528,  864,  251,  378,  442,
            ...
             681, 1389, 1488, 1049, 1330,  854, 1133,  398,  602, 1132],
           dtype='int64', length=896)

In [67]:
X_train = X.loc[train_index,]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([  3,   5,  10,  13,  14,\n            ...\n            888, 889, 891, 892, 894],\n           dtype='int64', length=278). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

Then I went to see the scikit-learn class defined for cross validation :

In [ ]:
class BaseCrossValidator(metaclass=ABCMeta):
    """Base class for all cross-validators
    Implementations must define `_iter_test_masks` or `_iter_test_indices`.
    """
    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            The target variable for supervised learning problems.
        groups : array-like of shape (n_samples,), default=None
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        X, y, groups = indexable(X, y, groups)
        indices = np.arange(_num_samples(X))
        for test_index in self._iter_test_masks(X, y, groups):
            train_index = indices[np.logical_not(test_index)]
            test_index = indices[test_index]
            yield train_index, test_index

    # Since subclasses must implement either _iter_test_masks or
    # _iter_test_indices, neither can be abstract.
    def _iter_test_masks(self, X=None, y=None, groups=None):
        """Generates boolean masks corresponding to test sets.
        By default, delegates to _iter_test_indices(X, y, groups)
        """
        for test_index in self._iter_test_indices(X, y, groups):
            test_mask = np.zeros(_num_samples(X), dtype=bool)
            test_mask[test_index] = True
            yield test_mask

    def _iter_test_indices(self, X=None, y=None, groups=None):
        """Generates integer indices corresponding to test sets."""
        raise NotImplementedError

    @abstractmethod
    def get_n_splits(self, X=None, y=None, groups=None):
        """Returns the number of splitting iterations in the cross-validator"""

    def __repr__(self):
        return _build_repr(self)